In [2]:
import mysql.connector
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

c:\Users\2ndro\anaconda3\envs\nlp-thesis-similarity\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
database = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
)
cursor = database.cursor()
cursor.execute("USE nlp_thesis_similarity")

In [4]:
model = SentenceTransformer("all-MiniLM-L6-v2")

In [5]:
# # First, make sure the columns exist
# try:
#     # Check if columns exist first
#     cursor.execute("SHOW COLUMNS FROM dewey_papers LIKE 'title_embeddings_all-MiniLM-L6-v2'")
#     if not cursor.fetchone():
#         cursor.execute("ALTER TABLE dewey_papers ADD COLUMN `title_embeddings_all-MiniLM-L6-v2` JSON")
        
#     cursor.execute("SHOW COLUMNS FROM dewey_papers LIKE 'abstract_embeddings_all-MiniLM-L6-v2'")
#     if not cursor.fetchone():
#         cursor.execute("ALTER TABLE dewey_papers ADD COLUMN `abstract_embeddings_all-MiniLM-L6-v2` JSON")
        
#     database.commit()
#     print("Columns checked and added if needed")
# except Exception as e:
#     print(f"Error managing columns: {e}")
#     database.rollback()  # Rollback in case of error

In [6]:
import json

def get_embedding(text, model):
    # Handle None values or empty strings
    if text is None or text == "":
        return None
    # Convert text to embedding using the provided model
    return model.encode(text).tolist()

def update_embeddings_in_batches(model, batch_size=100):
    try:
        # First, retrieve all papers that need embeddings
        cursor.execute("""
            SELECT id, title, abstract FROM dewey_papers 
            WHERE `title_embeddings_all-MiniLM-L6-v2` IS NULL 
            OR `abstract_embeddings_all-MiniLM-L6-v2` IS NULL
        """)
        all_papers = cursor.fetchall()
        
        print(f"Processing {len(all_papers)} papers that need embeddings...")
        
        if not all_papers:
            print("No papers need embeddings. Process complete.")
            return
            
        # Process in batches to avoid memory issues
        total_batches = (len(all_papers) + batch_size - 1) // batch_size
        
        for i in tqdm(range(0, len(all_papers), batch_size), total=total_batches, desc="Processing batches"):
            batch = all_papers[i:i+batch_size]
            updates = []
            
            for paper_id, title, abstract in tqdm(batch, leave=False, desc="Papers in batch"):
                title_embedding = get_embedding(title, model)
                abstract_embedding = get_embedding(abstract, model)
                
                if title_embedding is not None or abstract_embedding is not None:
                    updates.append((
                        json.dumps(title_embedding) if title_embedding else None, 
                        json.dumps(abstract_embedding) if abstract_embedding else None, 
                        paper_id
                    ))
            
            # Check if we have any updates to make
            if updates:
                # Update the database with the new embeddings
                update_query = """
                UPDATE dewey_papers 
                SET `title_embeddings_all-MiniLM-L6-v2` = %s, `abstract_embeddings_all-MiniLM-L6-v2` = %s 
                WHERE id = %s
                """
                cursor.executemany(update_query, updates)
                database.commit()
                print(f"Updated embeddings for {len(updates)} papers in batch")
                
    except Exception as e:
        print(f"Error during embedding update: {e}")
        database.rollback()
        raise

# Run the update process with the loaded model
update_embeddings_in_batches(model)

print("Embedding process completed")

Processing 7463 papers that need embeddings...


Processing batches:   1%|▏         | 1/75 [00:01<01:21,  1.10s/it]

Updated embeddings for 100 papers in batch


Processing batches:   3%|▎         | 2/75 [00:02<01:16,  1.05s/it]

Updated embeddings for 100 papers in batch


...

Updated embeddings for 100 papers in batch


Processing batches:  99%|█████████▊| 74/75 [02:55<00:03,  3.51s/it]

Updated embeddings for 100 papers in batch


Processing batches: 100%|██████████| 75/75 [02:57<00:00,  2.37s/it]

Updated embeddings for 63 papers in batch
Embedding process completed
